# Tarea 02
## Marco Aurelio Muñiz Lescale
### Representación gráfica de datos

Generar una visualización de un mapa con burbujas que permita visualizar las muertes de mujeres causadas por violecia familiar en México para el año 2022, para los datos totaless y por casa 100,000 habitantes.

## Preprocesamiento de los datos

In [1]:
# biblioteca
import pandas as pd

In [2]:
# lectura de datos
# Especifica las columnas que deseas leer
columnas_deseadas = ['ent_resid','causa_def','sexo','par_agre']

data = pd.read_csv('conjunto_de_datos_defunciones_registradas_2022_csv/conjunto_de_datos_defunciones_registradas_2022_csv/conjunto_de_datos/conjunto_de_datos_defunciones_registradas_2022.csv',usecols=columnas_deseadas)

In [3]:
# Primero veamos las columnas que sse utilizarán
data.head()

,ent_resid,causa_def,sexo,par_agre
0,1,K254,2,88
1,1,J439,2,88
2,1,E112,1,88
3,1,I269,1,88
4,1,W104,1,88


Para identificar la condición de violecia familiar se deben considerar los registros que tengan en el campo Causa de defunción (CAUSA_DEF) el rango X850-Y099, así como el Parentesco del presunto agresor (PAR_AGRE). El campo PAR_AGRE permite identificar la condición de violencia famialiar en las defunciones por presunto homicidio; los códigos del 01 al 26 correnponden a Sí hubo violencia familiar

In [4]:
# Veamos el tipo de dato en cada columna
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847716 entries, 0 to 847715
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ent_resid  847716 non-null  int64 
 1   causa_def  847716 non-null  object
 2   sexo       847716 non-null  int64 
 3   par_agre   847716 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 22.6+ MB


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847716 entries, 0 to 847715
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ent_resid  847716 non-null  int64 
 1   causa_def  847716 non-null  object
 2   sexo       847716 non-null  int64 
 3   par_agre   847716 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 22.6+ MB


In [6]:
# filtro de PAR_AGRE
mask1 = data['par_agre'].isin(list(range(1,27))+list(range(37,53)))
# filtro de sexo
mask2 = data['sexo'] == 2
data_filtered = data[mask1 & mask2]

In [7]:
# notamos que causa_def ya se encuentra en el rango 
data_filtered['causa_def'].unique()

array(['X910', 'X994', 'Y061', 'X990', 'X954', 'X950', 'X959', 'Y094',
       'Y044', 'Y090', 'X999', 'X958', 'X917', 'X914', 'Y099'],
      dtype=object)

In [8]:
data_filtered[['ent_resid','sexo']].groupby(by='ent_resid').count().reset_index().head()

,ent_resid,sexo
0,1,1
1,3,2
2,5,1
3,6,1
4,7,4


In [9]:
# para determinar la población y el nombre del estado
estado_df = pd.read_csv('conjunto_de_datos_defunciones_registradas_2022_csv/conjunto_de_datos_defunciones_registradas_2022_csv/catalogos/entidad_municipio_localidad_2022.csv')

In [10]:
mask1 = estado_df['cve_ent'].isin(range(1,33))
mask2 = estado_df['cve_loc'] == 0
mask3 = estado_df['cve_mun'] == 0
estado_df[mask1 & mask2 & mask3][['cve_ent','nom_loc']].head()

,cve_ent,nom_loc
0,1,Aguascalientes
189,2,Baja California
358,3,Baja California Sur
423,4,Campeche
651,5,Coahuila de Zaragoza


In [11]:
# poblacion
habitantes = pd.read_csv('Número de habitantes en México en 2023, por entidad federativa.csv')

In [12]:
habitantes.head()

,Entidad Federativa,Población (2022)
0,Aguascalientes,"1,424,542"
1,Baja California,"3,763,224"
2,Baja California Sur,"813,344"
3,Campeche,"953,450"
4,Chiapas,"5,369,105"


In [13]:
# se hace el join de las tablas de poblacio, localidad y homicidios
tabla_abt = estado_df[mask1 & mask2 & mask3][['cve_ent','nom_loc']].merge(
        data_filtered[['ent_resid','sexo']].groupby(by='ent_resid').count().reset_index(),how='inner',left_on='cve_ent',right_on='ent_resid').merge(habitantes,how='inner',left_on='nom_loc',right_on='Entidad Federativa')[['sexo','Entidad Federativa','Población (2022)']]

In [14]:
tabla_abt.columns =  ['proporcion','Entidad Federativa','Población (2022)']

In [15]:
# Reemplazar las comas por vacio
tabla_abt['Población (2022)'] = tabla_abt['Población (2022)'].str.replace(',', '').astype(float)

In [16]:
tabla_abt['tasa'] = (tabla_abt['proporcion'] / tabla_abt['Población (2022)'])*100000

In [17]:
coordenadas = pd.read_csv('Estidades Federativas de México y Coordendas de capitales.csv',usecols=['Estado','Latitud','Longitud'])

In [18]:
coordenadas.head()

,Estado,Latitud,Longitud
0,Aguascalientes,21.85,-102.30
1,Baja California,32.65,-115.50
2,Baja California Sur,24.13,-110.38
3,Campeche,19.83,-90.53
4,Chiapas,16.75,-93.12


In [19]:
tabla_abt = tabla_abt.merge(coordenadas,how='inner',left_on='Entidad Federativa',right_on='Estado')[['proporcion','Entidad Federativa','Población (2022)','tasa','Latitud','Longitud']]

In [20]:
# Finalmente, la tabla con la que trabajaremos es la siguiente
tabla_abt.head()

,proporcion,Entidad Federativa,Población (2022),tasa,Latitud,Longitud
0,1,Aguascalientes,1424542.0,0.070198,21.85,-102.30
1,2,Baja California Sur,813344.0,0.245898,24.13,-110.38
2,1,Coahuila de Zaragoza,3146792.0,0.031778,25.42,-101.00
3,1,Colima,751555.0,0.133057,19.22,-103.73
4,4,Chiapas,5369105.0,0.074500,16.75,-93.12


## Visualización

In [21]:
# Se utiliza la biblioteca de pytho plotly para la generación del mapa
import plotly.express as px
fig = px.scatter_mapbox(tabla_abt, lat="Latitud", lon="Longitud", color="tasa", size="proporcion", hover_name="Entidad Federativa",
                        color_continuous_scale="Viridis", size_max=20)


fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=4, margin={"r":15, "t":45, "l":15, "b":15})

fig.update_layout(title={
    "text": "Defunciones por violencia familiar en mujeres (proporción vs tasa por 100mil habitantes)",
    "font": {"size": 16, "family": "Arial"}
})

### Mapa

In [22]:
# Se utiliza el framework DASH para que se pueda visualizar en una archivo html
import dash
from dash import dcc, html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="mapa", figure=fig),
])

if __name__ == "__main__":
    app.run_server(debug=False)

